<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-3-Context-Engineering-for-Agentic-AI-Systems/M3LC3_Build_a_Multi_User_Conversational_Agentic_AI_Financial_Analyst_with_Memory_Context_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Multi-User Conversational Financial Analyst Tool-Use AI Agent with In-Memory Persistence with LangGraph

This project will focus on building a Tool-Use Agentic AI System which acts as a Financial Analyst & Advisor. This agent will be conversational and can handle multiple user-sessions with their own separate conversational history.

The first project here will build a conversational tool-use agentic system with in-memory persistence.

![](https://i.imgur.com/2xSAz1e.png)

### Financial Analyst Tool-Use Agentic AI System with In-Memory Persistence

This project focuses on building a **Tool-Use Agentic AI System** that acts as a Financial Analyst & Advisor. The agent will be conversational and can handle multiple user sessions with their own separate conversational history. By leveraging the `create_react_agent` function from **LangGraph**, this project adds in-memory persistence for enhanced user interaction continuity. The workflow comprises the following components:

1. **Agent System Prompt**:
   - The agent validates input queries for relevance and specificity.
   - It provides detailed market analysis or stock-specific insights depending on the user's query.
   - For invalid queries, the agent responds professionally and guides the user appropriately.
   - The system delivers concise, professional responses emphasizing data clarity and accuracy.

   **Flows**:
   - **Flow 1**: For general market trends, the agent analyzes data and suggests stock opportunities using tools like `SEARCH_WEB` and `GET_GENERAL_MARKET_DATA`.
   - **Flow 2**: For stock-specific queries, the agent validates the stock ticker, retrieves relevant data, and provides insights using tools such as `GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS` and `GET_STOCK_PRICE_METRICS`.

2. **Financial Analysis Tools**:
   The system integrates multiple tools to ensure comprehensive and precise insights:
   - **GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS**: Provides insights into key financial metrics such as P/E ratio, ROE, etc.
   - **GET_STOCK_NEWS**: Extracts the latest news and updates related to stocks or markets.
   - **GET_GENERAL_MARKET_DATA**: Fetches data on overall market trends and performance.
   - **GET_STOCK_TICKER**: Validates and fetches stock ticker symbols based on user queries.
   - **GET_STOCK_PRICE_METRICS**: Retrieves price trends, performance, and metrics for specific stocks.

3. **Stock Market Data Providers**:
   The system ensures real-time, reliable data by integrating with top providers like Yahoo Finance, Finviz, TMX, Cboe, and more, through platforms such as **OpenBB**.

4. **Memory Module**:
   - A memory module is introduced to store and manage user-specific conversational histories.
   - This enables the agent to maintain session continuity and provide context-aware responses for recurring users or prolonged interactions. __Here we store this in-memory.__

5. **ReAct Reasoning Framework**:
   - The system employs **ReAct reasoning**, combining logical deduction with dynamic tool usage. This framework ensures precise and actionable results by dynamically selecting the right tools based on the query.

6. **Final Response**:
   - After processing the data from tool calls, the agentic system generates the final response.






## Install OpenAI, LangGraph and LangChain dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.30 langgraph==0.6.5 langchain-community==0.3.27 langgraph-checkpoint-sqlite==2.0.11 --quiet

## Install OpenBB

In [ ]:
!pip install openbb[all]==4.4.5 --quiet

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter OpenBB Key

Get a free API key from [here](https://my.openbb.co/app/platform/pat)

In [ ]:
from getpass import getpass

OPENBB_PAT = getpass('Enter OpenBB Personal Access Token (PAT): ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [ ]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [ ]:
from openbb import obb
# takes 1 min to setup
obb.account.login(pat=OPENBB_PAT)

In [ ]:
FMP_API_KEY = getpass('Enter Financial Modeling Prep Key (FMP key): ')
obb.user.credentials.fmp_api_key = FMP_API_KEY

In [ ]:
obb.user.credentials

## Create Financial Tools

**Financial Analysis Tools**:
   The system integrates multiple tools to get useful financial data and metrics:

   - **GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS**: Provides insights into key financial metrics such as P/E ratio, ROE, etc.
   - **GET_STOCK_NEWS**: Extracts the latest news and updates related to stocks or markets.
   - **GET_GENERAL_MARKET_DATA**: Fetches data on overall market trends and performance.
   - **GET_STOCK_TICKER**: Validates and fetches stock ticker symbols based on user queries.
   - **GET_STOCK_PRICE_METRICS**: Retrieves price trends, performance, and metrics for specific stocks.

In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

tavily_search = TavilySearchAPIWrapper()


@tool
def get_stock_ticker_symbol(stock_name: str) -> str:
    """Look up a company's stock identifier.

    Args:
        stock_name: Company name or partial name to search.

    Returns:
        A markdown-formatted table with potential matches and identifiers
        (e.g., ticker, company name, CIK).

    Notes:
        - Uses OpenBB `equity.search` with provider "sec".
        - Results may include multiple matches for ambiguous names.
    """
    # Use OpenBB to search for stock ticker symbol and company details by name.
    # The provider "sec" fetches data from the U.S. Securities and Exchange Commission (SEC).
    try:
        res = obb.equity.search(stock_name, provider="sec")

        # Convert the result to a DataFrame and format it as markdown for readability.
        stock_ticker_details = res.to_df().to_markdown()

        # Prepare the output with the stock details.
        output = """Here are the details of the company and its stock ticker symbol:\n\n""" + stock_ticker_details
    except Exception as e:
        output = (
            "Please broaden your search and try again. "
            "Error encountered for search with query: " + stock_name +
            f"\n\nError details: {str(e)}"
        )
    return output


@tool
def get_stock_price_metrics(stock_ticker: str) -> str:
    """Retrieve quote, performance stats, and 1-year price history for a ticker.

    Args:
        stock_ticker: Exchange ticker symbol (e.g., "AAPL").

    Returns:
        A markdown-formatted string containing:
          - Price Quote Metrics (provider: cboe)
          - Price Performance Metrics (provider: finviz)
          - Daily Historical Prices for the past year (provider: yfinance)

    Notes:
        - Data sources have different refresh cadences and may lag real time.
        - Historical range is computed from the current date minus 365 days.
    """
    # Fetch the latest stock price quote using "cboe" provider.
    res = obb.equity.price.quote(stock_ticker, provider='cboe')
    price_quote = res.to_df().to_markdown()

    # Retrieve stock price performance metrics (e.g., percentage change) using "finviz" provider.
    res = obb.equity.price.performance(symbol=stock_ticker, provider='finviz')
    price_performance = res.to_df().to_markdown()

    # Fetch historical price data for the past year using "yfinance" provider.
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=365)).strftime("%Y-%m-%d")
    res = obb.equity.price.historical(symbol=stock_ticker, start_date=start_date,
                                      interval='1d', provider='yfinance')
    price_historical = res.to_df().to_markdown()

    # Combine the results into a formatted output.
    output = ("""Here are the stock price metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Price Quote Metrics:\n\n" + price_quote +
              "\n\nPrice Performance Metrics:\n\n" + price_performance +
              "\n\nPrice Historical Data:\n\n" + price_historical)
    return output


@tool
def get_stock_fundamental_indicator_metrics(stock_ticker: str) -> str:
    """Fetch annual fundamental metrics for a company.

    Args:
        stock_ticker: Exchange ticker symbol (e.g., "MSFT").

    Returns:
        A markdown-formatted table of key fundamental ratios and metrics for up to the last
        10 annual periods.

    Notes:
        - Uses OpenBB `equity.fundamental.metrics` with provider "yfinance" and "fmp.
        - Metric availability varies by company and period.
    """
    # Retrieve fundamental financial ratios (e.g., P/E ratio, ROE) using "fmp" provider.
    res = obb.equity.fundamental.ratios(symbol=stock_ticker, period='annual',
                                        limit=10, provider='fmp')
    fundamental_ratios = res.to_df().to_markdown()

    # Fetch additional fundamental metrics (e.g., EBITDA, revenue growth) using "yfinance" provider.
    res = obb.equity.fundamental.metrics(symbol=stock_ticker, period='annual',
                                         limit=10, provider='yfinance')
    fundamental_metrics = res.to_df().to_markdown()

    # Combine fundamental ratios and metrics into a single output.
    output = ("""Here are the fundamental indicator metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Fundamental Ratios:\n\n" + fundamental_ratios +
              "\n\nFundamental Metrics:\n\n" + fundamental_metrics)
    return output


@tool
def get_stock_news(stock_ticker: str, query: str) -> str:
    """Collect recent company news and augment with web results.

    Args:
        stock_ticker: Exchange ticker symbol (e.g., "NVDA").
        query: Free-text query about company name to guide additional web search context.

    Returns:
        A markdown-formatted string with:
          - Headlines from the last 45 days for the given ticker (provider: yfinance)
          - A short set of detailed articles from Tavily web search

    Notes:
        - Provider fields differ by source; this tool normalizes to title and text where possible.
        - Tavily results include raw content when available.
    """
    # Define the date range to fetch news (last 45 days).
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=45)).strftime("%Y-%m-%d")

    # Retrieve news headlines for the stock using "tmx" provider. - Changed to yfinance as tmx right now is not sending news for some sources
    res = obb.news.company(symbol=stock_ticker, start_date=start_date, provider='yfinance', limit=100)
    news = res.to_df()

    # Extract relevant columns (symbols and titles) and format as markdown.
    news = news[['title', 'text']]  # change column names based on provider - for tmx it will be symbol and text
    news['symbol'] = stock_ticker
    news = news.to_markdown()

    search_query = 'Recent news about ' + stock_ticker + ' ' + query
    results = tavily_search.raw_results(query=query,
                                        max_results=5,
                                        search_depth='advanced',
                                        include_answer=False,
                                        include_raw_content=True)
    results = tavily_search.raw_results(query=query, max_results=3, search_depth='advanced',
                                        include_answer=False, include_raw_content=True)
    docs = results['results']
    docs = [doc for doc in docs if doc.get("raw_content") is not None]
    docs = ['## Title\n' + doc['title'] + '\n\n' + '## Content\n' + doc['raw_content'] + '\n\n' + '##Source\n' + doc['url'] for doc in docs]
    detailed_news = '\n-----\n'.join(docs)

    # Prepare the output with the news headlines.
    output = ("""Here are the recent news for the stock ticker symbol """ + stock_ticker +
              """\n\nHeadlines: \n\n""" + news +
              """\n\nDetailed News Articles:\n\n""" + detailed_news
              )
    return output


@tool
def get_general_market_data() -> str:
    """Provide a quick market overview.

    Returns:
        A markdown-formatted summary of:
          - Most actively traded stocks by volume (top 15)
          - Top price gainers (top 15)
          - Top price losers (top 15)

    Notes:
        - Uses OpenBB equity discovery endpoints with provider "yfinance".
        - Lists are sorted in descending order by the provider's default metric.
    """
    # Retrieve the most actively traded stocks using "yfinance" provider.
    res = obb.equity.discovery.active(sort='desc', provider='yfinance', limit=15)
    most_active_stocks = res.to_df().to_markdown()

    # Fetch the top price gainers using "yfinance" provider.
    res = obb.equity.discovery.gainers(sort='desc', provider='yfinance', limit=15)
    price_gainers = res.to_df().to_markdown()

    # Retrieve the top price losers using "yfinance" provider.
    res = obb.equity.discovery.losers(sort='desc', provider='yfinance', limit=15)
    price_losers = res.to_df().to_markdown()

    # Combine the market data into a single formatted output.
    output = ("""Here's some detailed information of the stock market which includes most actively traded stocks, gainers and losers:\n\n""" +
              "Most actively traded stocks:\n\n" + most_active_stocks +
              "\n\nTop price gainers:\n\n" + price_gainers +
              "\n\nTop price losers:\n\n" + price_losers)
    return output

## Build the Conversational ReAct Agentic AI System with Transient In-Memory Store

Here we use an in-memory store for persisting conversational messages between the user and the agent temporarily (transient). This is in-memory so it would be deleted once you shut down the agent or your system.

### Add System Instruction Prompt

In [ ]:
AGENT_SYS_PROMPT = r"""
Role:
You are an AI Stock Market Assistant that provides investors with accurate, timely, and concise information on individual stocks or actionable insights from general market data.

Objective:
Help data-driven investors make informed decisions by delivering complete, relevant, and clearly formatted answers about stocks or market trends, strictly based on verified data retrieved from the available tools.

Available Tools:
- get_stock_ticker_symbol(company_name): Find ticker symbol(s), company name, and CIK for a given company.
- get_stock_price_metrics(ticker): Latest quote, performance stats, and 1-year price history.
- get_stock_fundamental_indicator_metrics(ticker): Key financial fundamentals such as P/E ratio, EBITDA, revenue growth, ROE.
- get_stock_news(ticker, query): Latest news headlines and detailed articles for the company.
- get_general_market_data(): Overview of most active stocks, top gainers, and top losers.

Starting Flow:
1. Input Validation:
   - If the query is about general market trends or stocks worth monitoring → Flow 1.
   - If the query is about a specific company or ticker → Flow 2.
   - Otherwise, respond politely that you only provide information based on stock market data and trends.

Flow 1: General Market Analysis
A. If the user wants general market insight or stock ideas:
   - Use get_general_market_data for top movers and active stocks.
   - Optionally, use get_stock_news for notable events affecting these stocks.
   - Summarize with key trends, highlighting noteworthy gainers/losers.

Flow 2: Company-Specific Query
A. Symbol Extraction:
   - If user gives a company name, call get_stock_ticker_symbol().
   - If no match, attempt name correction (e.g., "microsfot" → "microsoft")
      or broaden search (e.g., "google" → "alphabet").
   - If ticker/company still unclear, ask for clarification.
B. Data Retrieval:
   - Identify the type of information requested:
     -> Fundamentals, company performance → get_stock_fundamental_indicator_metrics
     -> Price trends, quotes, stock performance → get_stock_price_metrics
     -> News or events → get_stock_news
   - For general company overview, use a combination of all the following:
     -> get_stock_price_metrics
     -> get_stock_fundamental_indicator_metrics
     -> get_stock_news
   - Only use data from the provided tools.

Response Generation:
- Always analyze the retrieved data before responding.
- Present data in a concise, friendly, and professional tone.
- For recommendations, clearly state that the user should do their own research before investing.
- Format the final answer in markdown.
- Escape special characters for correct markdown rendering (e.g., $25.5 → \$25.5).
- Always show what the user asks for unless it is a general query, do NOT show excess information not related to the question

Example:
User: "What is the PE ratio for Eli Lilly?"
Agent:
1. Use get_stock_ticker_symbol("Eli Lilly") → LLY
2. Use get_stock_fundamental_indicator_metrics("LLY") → Retrieve P/E ratio
3. Respond: "The P/E ratio for Eli Lilly (LLY) as of May 12, 2024, is 30."
"""

In [ ]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")
tokens = enc.encode(AGENT_SYS_PROMPT)
print(f"Number of tokens: {len(tokens)}")

### Create the Agent Graph

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tools = [get_stock_ticker_symbol,
         get_stock_price_metrics,
         get_stock_fundamental_indicator_metrics,
         get_stock_news,
         get_general_market_data]

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)



SYS_MSG = SystemMessage(content=AGENT_SYS_PROMPT)
def chatbot(state: State):
    current_state = state["messages"]
    state_with_instructions = [SYS_MSG] + current_state
    response = [llm_with_tools.invoke(state_with_instructions)]

    token_usage = response[0].usage_metadata
    print('-'*25, 'TOKEN USAGE', '-'*25)
    print("Token count (input context):", token_usage['input_tokens'])
    print("Token count (response):", token_usage['output_tokens'])
    print('-'*25, 'TOKEN USAGE', '-'*25)
    return {"messages": response}

graph_builder.add_node("agent", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "agent",
    tools_condition,
    ['tools', END]
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "agent")
graph_builder.set_entry_point("agent")

# add memory and compile agent
memory = MemorySaver()
financial_analyst_agent = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display, Markdown

display(Image(financial_analyst_agent.get_graph().draw_mermaid_png()))

## Run and Test Agent

In [ ]:
# get agent streaming utils
!gdown 1dSyjcjlFoZpYEqv4P9Oi0-kU2gIoolMB

In [ ]:
from agent_utils import format_message

def call_conversational_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"messages": [{"role": "user", "content": prompt}]},
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        if verbose:
            format_message(event["messages"][-1])

    print('\n\nFinal Response:\n')
    display(Markdown(event["messages"][-1].content))
    return event["messages"]

## Test the Agentic System with In-Memory Persistence

### Simulating User 1

In [ ]:
us_id = 'user001-session'
query = '''Are PE Ratio, ROE and Revenue Growth three very important metrics to understand a company's health?
           Explain them briefly please'''
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'get these for Amazon?'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'do the same for microsoft'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'which stock might be the better pick?'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

### Simulating User 2

In [ ]:
us_id = 'user002-session'
query = 'how is nvidia doing currently?'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'what about intel?'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'which is a better stock to pick?'
response = call_conversational_agent(agent=financial_analyst_agent,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

We have successfully built an AI Agent which can do detailed financial analysis for us and is completely conversational. Next up we will see how to use persistent memory on the disk

# Optimize your Conversational Financial Assistant Agentic AI System with Memory Context Engineering

This section optimizes the base agent to be better, cheaper, and more reliable by controlling context growth and normalizing tool outputs.  

We introduce memory-aware techniques that keep only the most relevant history, compress verbose tool results, and enforce tight token capping so the agent stays responsive even over long sessions and does not error out. This is depicted in the following architecture.

![](https://i.imgur.com/I1UPOlu.png)

Key optimizations:
- **Context Trimming**: Apply a recency message trimmer to retain only the most recent, agent memory history while discarding older history.
- **Tool Output Summarization**: Wrap tool executions with an LLM summarizer that converts long raw results into concise, structured summarized content.
- **Context Limit Guards**: Enforce upper bounds on message/tool payload size, and standardize return schemas to reduce downstream parsing errors.
- **Stateful External Memory**: Continue using a checkpointer and save the memory on the disk in a database so multi-turn sessions share context without exceeding the model’s context window.

These changes improve the overall agent in terms of being able to retain and leverage richer and longer context over multiple interactions without erroring out.


In [ ]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db*

## Create a mapping of `{tool name : actual tool implementation}`

Establish a canonical lookup between the LLM’s requested tool name and the executable tool function.  



In [ ]:
tools_by_name = {tool.name: tool for tool in tools}
tools_by_name

## Build the Conversational ReAct Agentic AI System with Persistent On-disk External Memory Store

Here we use a persistent SQLite database to permanently store our conversations between the agent and the user.

We will use `SqliteSaver` which helps to store separate conversation histories per user session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time. The memory is persisted on-disk so can be accessed anytime.



### Define Agent Node Functions & Graph

Refactor the graph to include trimming and summarization steps in the control flow.

Graph outline:
- **Context Trimmer Node**: Prunes history to a target token budget before each LLM turn.
- **Agent (LLM) Node**: Reads system prompt + trimmed history; decides whether to answer or call a tool.
- **Tool+Summarizer Node**: Executes the selected tool via `tools_by_name`, then immediately summarizes/normalizes the results and appends a compact `ToolMessage`.
- **Router**: If more info is needed, loop back to the Agent; otherwise transition to `END`.


In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.messages import trim_messages
from langgraph.graph.message import RemoveMessage, REMOVE_ALL_MESSAGES
from langchain_core.messages import ToolMessage


class State(TypedDict):
    messages: Annotated[list, add_messages]


tools = [get_stock_ticker_symbol,
         get_stock_price_metrics,
         get_stock_fundamental_indicator_metrics,
         get_stock_news,
         get_general_market_data]

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
small_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)
SYS_MSG = SystemMessage(content=AGENT_SYS_PROMPT)

TOOL_SUMMARIZER_PROMPT = """Given the following context information,
summarize it ensuring to retain all relevant / essential information.
Your goal is simply to reduce the size of the context to a more manageable size.
Follow these exact rules for summarization:
- Create a comprehensive report of approx 50000 words max
- Keep any critical facts or events in the summary
"""

def tool_node_with_summarization(state: State):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        # call the tool
        # print('calling tool', tool_call["name"], 'with args:', tool_call["args"])
        tool = tools_by_name[tool_call["name"]]
        tool_results = tool.invoke(tool_call["args"])
        # print(tool_results)
        if tool_call['name'] not in ['get_stock_news']:
            # print('non summ entering for ', tool_call['name'])
            result.append(ToolMessage(content=tool_results,
                                      tool_call_id=tool_call["id"]))
        else:
            # print('Entering for ', tool_call['name'])
            if type(tool_results) == list:
                tool_results = [str(doc) for doc in tool_results]
            else:
                tool_results = [str(tool_results)]
            # trim tool response context to < LLM max context window
            trimmed_tool_results = trim_messages(
                tool_results,
                max_tokens=120000,              # GPT-4.1-mini supports up to ~1M tokens
                strategy="first",                # Retain the most recent messages
                token_counter=ChatOpenAI(model="gpt-4o-mini"),  # Use LLM-based token counting
                allow_partial=True              # Allow partial trimming of messages if needed
            )
            print('-'*25, 'Summarizing Tool Results', '-'*25)
            # Summarize the tool response context further
            # You can handle various types of tool calls in different ways in this node
            summarized_tool_results = small_llm.invoke([{"role":"system", "content":TOOL_SUMMARIZER_PROMPT},
                                                        {"role":"user", "content" : str(trimmed_tool_results)}])
            # tokens = enc.encode(summarized_tool_results.content)
            # print(f"summ Number of tokens: {len(tokens)}")
            # add transformed tool response context as tool message in agent state
            result.append(ToolMessage(content=summarized_tool_results.content,
                                    tool_call_id=tool_call["id"]))
    return {"messages": result}


def context_trimmer_node(state: State) -> State:
    # Trim the context history to fit within the model's token limit
    # can summarize the whole past context also except last interaction if needed
    print('-'*25, 'Trimming overall context if needed', '-'*25)
    trimmed_state = trim_messages(
        state["messages"],
        max_tokens=200,     # last ~50 interactions approx - 1 agent interaction here is approx 4+ messages (input, tool call, tool result, reponse)
        strategy="last",    # Retain the most recent messages
        token_counter=len,  # Use number of state messages based counting
        allow_partial=True  # Allow partial trimming of messages if needed
    )

    return {"messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)] + trimmed_state}

# Create the node function that handles reasoning and planning using the LLM
def tool_calling_llm(state: State) -> State:
    # get agent state and then get next step (response) from llm
    current_state = state["messages"]
    state_with_instructions = [AGENT_SYS_PROMPT] + current_state
    response = [llm_with_tools.invoke(state_with_instructions)]

    token_usage = response[0].usage_metadata
    print('-'*25, 'TOKEN USAGE', '-'*25)
    print("Token count (input context):", token_usage['input_tokens'])
    print("Token count (response):", token_usage['output_tokens'])
    print('-'*25, 'TOKEN USAGE', '-'*25)

    # Return the updated state containing the new message
    return {"messages": response}

# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def tool_calling_routing(state: State) -> str:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_call"
    # Otherwise, we stop (reply to the user)
    return "stop_agent"

# Build the graph
builder = StateGraph(State)

# Add nodes
builder.add_node("context_trimmer", context_trimmer_node)
builder.add_node("agent", tool_calling_llm)
builder.add_node("tools_with_summarizer", tool_node_with_summarization)

# Add edges
builder.add_edge(START, "context_trimmer")
# Conditional edge
builder.add_conditional_edges(
    "agent",
    tool_calling_routing, # conditional routing function
    {
        "stop_agent": END,
        "tool_call": "tools_with_summarizer"
    }
)
builder.add_edge("tools_with_summarizer", "context_trimmer") # this is the key feedback loop in the agentic system
builder.add_edge("context_trimmer", "agent")
# Compile Agent Graph
financial_analyst_agent = builder.compile()

In [ ]:
from IPython.display import Image, display, Markdown

display(Image(financial_analyst_agent.get_graph().draw_mermaid_png()))

## Run and Test Agent

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

# remember to send the agent graph and not the compiled agent to this function
def call_conversational_agent(agent_graph, prompt, user_session_id, verbose=False):
    with SqliteSaver.from_conn_string("memory.db") as memory:
        agent = agent_graph.compile(checkpointer=memory)
        events = agent.stream(
            {"messages": [{"role": "user", "content": prompt}]},
            {"configurable": {"thread_id": user_session_id}},
            stream_mode="values",
        )

        print('Running Agent. Please wait...')
        for event in events:
            if verbose:
                format_message(event["messages"][-1])

        print('\n\nFinal Response:\n')
        display(Markdown(event["messages"][-1].content))
        return event["messages"]

### Test the Agentic System with On-Disk Persistence & Memory Context Engineering

Let's now simulate User 1 using the agent

In [ ]:
us_id = 'user001-session'
query = 'show me the health of companies amazon and microsoft based on their fundamentals and price trends'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'do the same for amazon and meta'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=uid,
                                     verbose=False)

In [ ]:
query = 'do a comparative analysis of these stocks based on the data and tell me what could be a good investment'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

Let's now simulate User 2 using the agent

In [ ]:
us_id = 'user002-session'
query = 'give me an overall report of how is nvidia doing as company'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

In [ ]:
query = 'do the same for intel'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=False)

In [ ]:
query = 'which has a better outlook for investing?'
response = call_conversational_agent(agent_graph=builder,
                                     prompt=query,
                                     user_session_id=us_id,
                                     verbose=True)

We can go on running this agent and the overall context window size maintained in the agent memory is reduced by **approximately 90%**

In [ ]:
((328907-34773) / 328907) * 100 # roughly 34K tokens are in memory after multiple turns